# Trace Visualization Demo

This notebook demonstrates how to use the trace visualization functions to compare expected and actual traces in instrumentation tests.

In [ ]:
# Import required modules
import sys
import os

# Add pywhy to path if needed
sys.path.append('..')

from pywhy.trace_visualization import (
    format_trace, compare_traces, display_trace_comparison, 
    show_trace_diff, print_trace_comparison
)
from pywhy.trace_dsl import trace
from pywhy.instrumenter import exec_instrumented
from pywhy.tracer import get_tracer

print("Modules imported successfully!")

## Example 1: Simple Assignment Trace Comparison

In [ ]:
# Create expected trace using DSL
expected_trace = (
    trace()
    .assign("x", 10)
    .assign("y", 20) 
    .assign("z", 30)
    .build()
)

print("Expected trace created:")
print(format_trace(expected_trace, "Expected Assignment Trace"))

In [ ]:
# Run actual instrumentation
tracer = get_tracer()  # Use global tracer
tracer.clear()  # Clear any previous events

code = """
x = 10
y = 20
z = x + y
"""

# Execute with instrumentation - the tracer will automatically collect events
exec_instrumented(code)
actual_events = tracer.events

print("Actual trace:")
print(format_trace(actual_events, "Actual Assignment Trace"))

In [ ]:
# Compare traces and show diff
comparison = compare_traces(actual_events, expected_trace)
print(display_trace_comparison(comparison))

In [ ]:
# Use Jupyter-specific display function
from IPython.display import HTML, display
from pywhy.trace_visualization import create_jupyter_trace_display

html_output = create_jupyter_trace_display(actual_events, expected_trace, "Assignment Test")
display(HTML(html_output))

## Example 2: Function Call Trace Comparison

In [ ]:
# Create expected trace for function calls
expected_function_trace = (
    trace()
    .function_entry("add_numbers", [5, 3])
    .assign("result", 8)
    .return_event(8)
    .assign("output", 8)
    .build()
)

print("Expected function trace:")
print(format_trace(expected_function_trace, "Expected Function Trace"))

In [ ]:
# Function code to instrument
function_code = """
def add_numbers(a, b):
    result = a + b
    return result

output = add_numbers(5, 3)
"""

# Use global tracer and clear previous events
tracer = get_tracer()
tracer.clear()

exec_instrumented(function_code)
actual_function_events = tracer.events

print("Actual function trace:")
print(format_trace(actual_function_events, "Actual Function Trace"))

In [ ]:
# Show function trace comparison
html_output = create_jupyter_trace_display(
    actual_function_events, expected_function_trace, "Function Call Test"
)
display(HTML(html_output))

## Example 3: Using Test-Attached Functions

The instrumentation tests now have trace comparison functions attached to them. Here's how to access them:

In [ ]:
# This would be the pattern for accessing test-attached functions
# (Requires running the actual tests first)

print("""
To use test-attached trace comparison functions:

1. Run a test:
   pytest -v tests/test_instrumentation.py::TestBasicInstrumentation::test_simple_assignment_instrumentation

2. The test instance will have these functions attached:
   - show_assignment_trace_comparison(): Display in Jupyter
   - get_assignment_trace_strings(): Get string representations
   - print_assignment_traces(): Print to console

3. Similar functions are available for other tests:
   - show_function_trace_comparison()
   - show_recursion_trace_comparison()
   - etc.
""")

## Helper Functions for Quick Access

In [ ]:
def create_and_compare_traces(code: str, expected_dsl_builder, test_name: str):
    """
    Helper function to create and compare traces for any code.
    
    Args:
        code: Python code to instrument and trace
        expected_dsl_builder: DSL builder for expected trace
        test_name: Name for the test
    """
    # Create expected trace
    expected_trace = expected_dsl_builder.build()
    
    # Use global tracer and clear previous events
    tracer = get_tracer()
    tracer.clear()
    
    # Run instrumentation
    exec_instrumented(code)
    actual_events = tracer.events
    
    # Show comparison
    html_output = create_jupyter_trace_display(actual_events, expected_trace, test_name)
    display(HTML(html_output))
    
    return actual_events, expected_trace

print("Helper function defined!")

In [ ]:
# Example usage of helper function
loop_code = """
total = 0
for i in range(3):
    total += i
"""

expected_loop = (
    trace()
    .assign("total", 0)
    .assign("i", 0)
    .aug_assign("total", 0)
    .assign("i", 1) 
    .aug_assign("total", 1)
    .assign("i", 2)
    .aug_assign("total", 3)
)

actual, expected = create_and_compare_traces(loop_code, expected_loop, "Loop Test")

## Summary

The trace visualization system provides:

1. **String formatting** of traces for readable output
2. **Diff generation** to show differences between expected and actual traces
3. **Jupyter-friendly HTML output** with side-by-side comparison
4. **Test-attached functions** that can be called from notebooks
5. **Helper functions** for quick trace comparison

This makes it easy to:
- Debug instrumentation issues
- Verify test correctness
- Understand trace execution patterns
- Create visual comparisons for documentation